#### Importing libraries 

In [430]:
from sklearn.pipeline import Pipeline
import sklearn
import pandas as pd
import numpy as np
from numpy import asarray
from datetime import datetime
import time
from sklearn import metrics
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
import locale
from yellowbrick.model_selection import learning_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer


#### Import data

In [431]:
df = pd.read_csv(r'C:\Users\hp\OneDrive\Documenten\Data Science Lab\listings_summary.csv')

## Data preprocessing

### Column-specific preprocessing

##### Drop features irrelevant to price, based on intuition

In [432]:
df = df.drop(['id', 'listing_url', 'host_neighbourhood', 'calendar_updated', 'host_picture_url', 
                                  'scrape_id', 'last_scraped', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
                                  'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url','calendar_last_scraped', 
                                  'license', 'jurisdiction_names', 'weekly_price', 'monthly_price'], axis = 1)


##### Assuming model must be globally applicable, we remove all location based features as it is only Berlin, Germany based

In [433]:
df = df.drop(['zipcode','street', 'neighbourhood', 'neighbourhood_cleansed', 
            'neighbourhood_group_cleansed', 'city' , 'state', 'market', 
            'smart_location', 'country_code', 'country', 'longitude', 'latitude'],axis = 1)


##### Drop 'free text' features

In [434]:
df = df.drop(['name', 'summary', 'space', 'description', 'experiences_offered', 
                                  'neighborhood_overview', 'notes', 'transit', 'transit', 'access', 
                                'interaction', 'house_rules'], axis = 1)

##### Drop features with a missing values percentage higher than 50% of the total amount of observations

In [435]:
missing_values_percentage = df.isnull().sum() / len(df)
selected_features = missing_values_percentage[missing_values_percentage > 0.5].index
print(selected_cols)
df = df.drop(selected_features, axis = 1)


Index(['host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'square_feet'],
      dtype='object')


##### 'host_listing_count' and 'host_total_listings_count' have only 26 non-similar values which are all NaN's, so we drop one of these features

In [436]:
print(sum((df.host_listings_count == df.host_total_listings_count) == False))
# = Listing_Data.loc[((Listing_Data.host_listings_count == Listing_Data.host_total_listings_count) == False)]
df = df.drop(['host_total_listings_count'], axis = 1)

26


##### Features 'host_verifications' and 'amenities' removed due to complexity i.c.w. time limitations

In [437]:
df = df.drop(['host_verifications', 'amenities'], axis = 1)

##### Features 'host_since', 'first_review' and 'last_review' are all datetimes which we transform to numerical 'years since' features making them more suitable for prediction. After transformation, we drop the original features 

In [438]:
df[['host_since','first_review', 'last_review']]  = df[['host_since', 'first_review', 'last_review']].applymap(pd.to_datetime)

df['host_years_active'] = (datetime.now() - df['host_since']).astype('timedelta64[Y]')
df['years_since_first_review'] = (datetime.now() - df['first_review']).astype('timedelta64[Y]')
df['years_since_last_review'] = (datetime.now() - df['last_review']).astype('timedelta64[Y]')

df = df.drop(['host_since', 'first_review', 'last_review'], axis = 1)

df[['host_years_active','years_since_first_review', 'years_since_last_review']].head()

,host_years_active,years_since_first_review,years_since_last_review
0,14.0,6.0,4.0
1,14.0,4.0,4.0
2,14.0,13.0,6.0
3,14.0,9.0,4.0
4,13.0,13.0,4.0


##### Transform 'host_location' into binary feature indicating whether host lives in city of property or not

In [439]:
df['host_in_city'] = df['host_location'].apply(lambda x:'t' if x == 'Berlin, Berlin, Germany' else 'f')
df = df.drop(['host_location'], axis = 1)
df['host_in_city'].head()

0    f
1    t
2    f
3    t
4    t
Name: host_in_city, dtype: object

##### For target variable 'price' and features 'security_deposit', 'cleaning_fee' and 'extra_people' remove $ and '.00' convert to float 

In [440]:
df[['price', 'security_deposit', 'cleaning_fee','extra_people']] = df[['price', 'security_deposit', 'cleaning_fee','extra_people']].apply(lambda x:x.str[1:-3])
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
df[['price', 'security_deposit', 'cleaning_fee','extra_people']] = df[['price', 'security_deposit', 'cleaning_fee','extra_people']].astype(str).applymap(locale.atof)

##### As the range of categories of feature 'property_type' is very broad and unequally distributed we assign all categories apart from 'House' and 'Apartment' to category 'Other'

In [441]:
print(df['property_type'].value_counts())
df.loc[~df['property_type'].isin(['House', 'Apartment']), 'property_type'] = 'Other'


Apartment                 20225
Condominium                 612
Loft                        460
House                       398
Serviced apartment          175
Hostel                      128
Townhouse                    99
Guest suite                  74
Bed and breakfast            64
Guesthouse                   57
Hotel                        50
Other                        47
Boutique hotel               43
Bungalow                     20
Boat                         17
Tiny house                   12
Houseboat                    11
Camper/RV                    11
Villa                        10
Aparthotel                    7
Pension (South Korea)         7
Cabin                         6
Cottage                       4
Resort                        3
Castle                        2
Casa particular (Cuba)        2
Train                         2
Tipi                          1
Chalet                        1
Barn                          1
Island                        1
In-law  

### Datatype-specific preprocessing (scaling, imputing)

##### Separate target variable 'price' from features

In [442]:
X = df.loc[:,df.columns != 'price']
y = df['price']

##### Perform train/test (80/20) split before datatype-specific preprocessing to avoid data leakage

In [443]:
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)

##### Distinguish numerical, binary and nominal features and perform completeness check

In [444]:
num_features = ['host_listings_count', 'host_years_active', 'years_since_first_review', 
                'years_since_last_review', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 
                'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
                'minimum_nights', 'maximum_nights','availability_30', 'availability_60', 
                'availability_90', 'availability_365', 'number_of_reviews', 'review_scores_rating', 
                'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
                'review_scores_communication', 'review_scores_location', 'review_scores_value', 
                'calculated_host_listings_count', 'reviews_per_month'] 
bin_features = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'is_location_exact',
                'has_availability', 'requires_license','instant_bookable', 'is_business_travel_ready',
                'require_guest_profile_picture', 'require_guest_phone_verification', 'host_in_city']
nom_features = ['property_type', 'room_type', 'bed_type','cancellation_policy']

all_features = num_features + bin_features + nom_features
completeness_check = len(X_train.drop(all_features, axis = 1).columns)
print(completeness_check)

0


##### Create customized encoder function that replaces 't' and 'f' into binary variables 1 respectively 0 

In [445]:
def binary_replacer(df):
    return df.replace({'t': 1, 'f': 0})

##### Create pipeline for each datatype: StandardScaler and KNNImputer (missing values) for numerical features, Encoder and Imputer for binary features and Encoder for nominal features

In [446]:
num_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('imputer', KNNImputer())])
bin_pipeline = Pipeline([('encoder', FunctionTransformer(binary_replacer)),
                 ('imputer', SimpleImputer(strategy = 'most_frequent'))])
nom_pipeline = Pipeline([('encoder', OneHotEncoder(sparse = False, drop = 'first'))])

##### Merge all transformer pipelines into one transformer pipeline, where each pipeline is only executed with the corresponding feature data with a specific datatype

In [447]:
combined_transformer = ColumnTransformer([
        ("numerical",num_pipeline, num_features),
        ("binary", bin_pipeline, bin_features),
        ("nominal",nom_pipeline, nom_features)])

### Instantiate ML algorithms

##### Extend pipeline with baseline model: Linear Regression and fit the training data

In [448]:
pipe_lin_regr = Pipeline([('preprocessing', combined_transformer), 
                     ('model', LinearRegression())])
lin_regr_fit = pipe_lin_regr.fit(X_train, y_train)

y_pred_lin_regr_test = lin_regr_fit.predict(X_test)
y_pred_lin_regr_train = lin_regr_fit.predict(X_train)

##### Extend pipeline with Random Forest Regression Model

In [449]:
pipe_rand_forest = Pipeline([('preprocessing', combined_transformer), 
                     ('model', RandomForestRegressor())])

##### Perform small hyperparameter optimization using a 5-fold cross-validated grid search over manually selected parameters

In [450]:
param_grid = [{
    'model__n_estimators': [50,100,150],
    'model__max_depth': [5, 10, 30],
    'model__max_features':['sqrt', 'log2']
}]

grid_cv_rand_forest = GridSearchCV(pipe_rand_forest, param_grid, scoring=make_scorer(mean_squared_error), cv=5)
grid_cv_rand_forest.fit(X_train, y_train)

##### Retrieve 'Best Parameters' and 'Best Score' and fit best model on training data

In [ ]:
print("Best Parameters: ", grid_cv_rand_forest.best_params_)
print("Best Score: ", grid_cv_rand_forest.best_score_)

y_pred_rand_forest_test = grid_cv_rand_forest.predict(X_test)
y_pred_rand_forest_train = grid_cv_rand_forest.predict(X_train)


Best Parameters:  {'model__max_depth': 5, 'model__max_features': 'log2', 'model__n_estimators': 50}
Best Score:  26491.67207229123


In [ ]:
rmse_lr_train = metrics.mean_squared_error(y_train, y_pred_lin_regr_train, squared = False)
r2_lr_train= metrics.r2_score(y_train, y_pred_lin_regr_train)
rmse_lr_test = metrics.mean_squared_error(y_test, y_pred_lin_regr_test, squared = False)
r2_lr_test = metrics.r2_score(y_test, y_pred_lin_regr_test)

rmse_rf_train = metrics.mean_squared_error(y_train, y_pred_rand_forest_train, squared = False)
r2_rf_train = metrics.r2_score(y_train, y_pred_rand_forest_train)
rmse_rf_test = metrics.mean_squared_error(y_test, y_pred_rand_forest_test, squared = False)
r2_rf_test = metrics.r2_score(y_test, y_pred_rand_forest_test)
print("LR Root Mean Squared Error Train: ", rmse_lr_train)
print("LR R-squared Train:", r2_lr_train)
print("LR Root Mean Squared Error Test: ", rmse_lr_test)
print("LR R-squared Test:", r2_lr_test)

print("RF Root Mean Squared Error Train: ", rmse_rf_train)
print("RF R-squared Train:", r2_rf_train)
print("RF Root Mean Squared Error Test: ", rmse_rf_test)
print("RF R-squared Test:", r2_rf_test)

LR Root Mean Squared Error Train:  232.59387774559005
LR R-squared Train: 0.06267049596699192
LR Root Mean Squared Error Test:  166.91826197446642
LR R-squared Test: -0.030920707588969698
RF Root Mean Squared Error Train:  140.0087885180471
RF R-squared Train: 0.6603697815653287
RF Root Mean Squared Error Test:  130.8568844029364
RF R-squared Test: 0.36640650914278394
